# Import Modules

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

import urllib3

In [ ]:
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인해 봅니다.
tf.config.list_physical_devices('GPU')

# Load Data

In [ ]:
urllib3.disable_warnings()

(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)

In [ ]:
type(ds_train)

In [ ]:
ds_info.features

In [ ]:
# 데이터의 개수도 확인해 봅시다. 
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

In [ ]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, [32, 32])
    image = tf.cast(image, tf.float32) / 255.
    return image, label

In [ ]:
def apply_normalize_on_dataset(ds, batch_size, is_test=False):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
#     ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
ds_info.features["label"].num_classes

In [ ]:
ds_info.features["label"].names

In [ ]:
fig = tfds.show_examples(ds_train, ds_info)

In [ ]:
fig = tfds.show_examples(ds_test, ds_info)


# Build Models

In [ ]:
def conv_layer(x, channel, kernel_size, block_num, cnn_num, i, strides):
    x = keras.layers.Conv2D(filters=channel, kernel_size=kernel_size, padding='same', activation=None, strides=strides, 
                        kernel_initializer='he_normal', name=f'stage{int(block_num+2)}_{int(cnn_num+1)}_conv{i}')(x)
    x = keras.layers.BatchNormalization(name=f'stage{block_num+2}_{cnn_num+1}_bn{i}')(x)
    return x

In [ ]:
def convolutional_block(input_layer, num_cnn, channel, block_num, is_50):
    x = input_layer
    ## ResNet34
    if is_50==False:
        kernel_size=(3,3)
        for i, cnn_num in enumerate(range(num_cnn)):
            skipconnection= x
            if i==0 and block_num>0:
                skipconnection= keras.layers.Conv2D(channel, (1, 1), strides=(2,2), use_bias=False, kernel_initializer='he_normal')(x)
                x = conv_layer(x, channel, kernel_size, block_num, cnn_num, 1, strides=2)
            else:
                x = conv_layer(x, channel, kernel_size, block_num, cnn_num, 1, strides=1)

            x = keras.layers.Activation('relu')(x)

            x = conv_layer(x, channel, kernel_size, block_num, cnn_num, 2, strides=1)
            x = keras.layers.Add(name=f'stage{block_num+2}_{cnn_num+1}_add')([x, skipconnection])
            x = keras.layers.Activation('relu')(x)

    ## ResNet50
    elif is_50==True:
        for i, cnn_num in enumerate(range(num_cnn)):
            skipconnection= x
            if i==0 and block_num==0:
                skipconnection= keras.layers.Conv2D(channel*4, (1, 1), strides=1, use_bias=False, kernel_initializer='he_normal')(x)
                x = conv_layer(x, channel, (1,1), block_num, cnn_num, 1, strides=1)
            elif i==0 and block_num>0:
                skipconnection= keras.layers.Conv2D(channel*4, (1, 1), strides=2, use_bias=False, kernel_initializer='he_normal')(x)
                x = conv_layer(x, channel, (1,1), block_num, cnn_num, 1, strides=2)
            elif i==1:
                x = conv_layer(x, channel, (1,1), block_num, cnn_num, 1, strides=1)
            elif i==2:
                x = conv_layer(x, channel*4, (1,1), block_num, cnn_num, 1, strides=1)

            x = keras.layers.Activation('relu')(x)

            x = conv_layer(x, channel, (3,3), block_num, cnn_num, 2, strides=1)
            x = keras.layers.Activation('relu')(x)
            
            x = conv_layer(x, (channel*4), (1,1), block_num, cnn_num, 9, strides=1)
            x = keras.layers.Add(name=f'stage{block_num+2}_{cnn_num+1}_add')([x, skipconnection])
            x = keras.layers.Activation('relu')(x)      
    return x

In [ ]:
def build_resnet(input_layer, is_50):
    num_cnn_list=[3, 4, 6, 3]
    channel_list=[64,128, 256, 512]
    x= input_layer
    
    #CONV1
    x = keras.layers.Conv2D(filters=64, kernel_size=(7,7), strides=2, padding='same', name=f'conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=(3,3), padding='same', strides=2, name=f'initial_maxpooling')(x)
    
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        x = convolutional_block(x, num_cnn, channel, i, is_50)
    
    x = keras.layers.AveragePooling2D(pool_size=(1,1), strides=None, padding='same', data_format=None,name='avg_pool')(x)
    x= keras.layers.Flatten()(x)
    x = keras.layers.Dense(1, activation='softmax', name='fc1000')(x)
    model= keras.Model(inputs= input_layer, outputs=x)
    return model


## ResNet-34

In [ ]:
input_shape=(32, 32,3)
input_layer = keras.layers.Input(shape=input_shape, name='input')  # input layer를 만들어둡니다.

resnet_34 = build_resnet(input_layer, is_50=False)
resnet_34.summary()

## ResNet-50

In [ ]:
input_shape=(32, 32,3)
input_layer = keras.layers.Input(shape=input_shape, name='input')  # input layer를 만들어둡니다.

resnet_50 = build_resnet(input_layer, is_50=True)
resnet_50.summary()

# TRAINING

In [ ]:
BATCH_SIZE = 32
EPOCH = 20

In [ ]:
ds_train_ = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test_ = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [ ]:
resnet_34.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=tf.keras.metrics.Accuracy(name='accuracy', dtype=None),
)

history_resnet_34 = resnet_34.fit(
    ds_train_,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test_,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
import pandas as pd

In [ ]:
df_resnet_34 = pd.DataFrame({
    'train_loss': history_resnet_34.history['loss'],
    'val_acc': history_resnet_34.history['val_accuracy']
    })
df_resnet_34.to_csv('./history_resnet_34.csv', index=False)

In [ ]:
resnet_50.compile(
#     loss='sparse_categorical_crossentropy',
#     optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=tf.keras.metrics.Accuracy(name='accuracy', dtype=None),
)

history_resnet_50 = resnet_50.fit(
    ds_train_,
    steps_per_epoch=np.ceil(ds_info.splits['train[:80%]'].num_examples),#/BATCH_SIZE),
    validation_steps=np.ceil(ds_info.splits['train[80%:]'].num_examples),#/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test_,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
df_resnet_50 = pd.DataFrame({
    'train_loss': history_resnet_50.history['loss'],
    'val_acc': history_resnet_50.history['val_accuracy']
    })
df_resnet_50.to_csv('./history_resnet_34.csv', index=False)

## data shape= (224, 224, 3)

In [ ]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, [224, 224])
    return tf.cast(image, tf.float32) / 255., label

In [ ]:
input_shape=(224, 224,3)
input_layer = keras.layers.Input(shape=input_shape, name='input')  # input layer를 만들어둡니다.

resnet_34 = build_resnet(input_layer, is_50=False)
resnet_34.summary()

In [ ]:
input_shape=(224, 224,3)
input_layer = keras.layers.Input(shape=input_shape, name='input')  # input layer를 만들어둡니다.

resnet_50 = build_resnet(input_layer, is_50=True)
resnet_50.summary()

In [ ]:
BATCH_SIZE = 128
EPOCH = 20

In [ ]:
ds_train_ = apply_normalize_on_dataset(ds_train, is_test, batch_size=BATCH_SIZE)
ds_test_ = apply_normalize_on_dataset(ds_test, is_test, batch_size=BATCH_SIZE)

In [ ]:
ds_info.splits['train[:80%]'].num_examples

In [ ]:
int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE)

In [ ]:
int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE)

In [ ]:
resnet_34.compile(
    loss='binary_crossentropy',
    optimizer="adam", # tf.keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy'],
)

history_resnet_34_224 = resnet_34.fit(
    ds_train_,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test_,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
df_resnet_34_224 = pd.DataFrame({
    'train_loss': history_resnet_34_224.history['loss'],
    'val_acc': history_resnet_34_224.history['val_accuracy']
    })
df_resnet_34_224.to_csv('./history_resnet_34_224.csv', index=False)

In [ ]:
resnet_50.compile(
#     loss='sparse_categorical_crossentropy',
#     optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy'],
)

history_resnet_50_224 = resnet_50.fit(
    ds_train_,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test_,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
df_resnet_50_224 = pd.DataFrame({
    'train_loss': history_resnet_50_224.history['loss'],
    'val_acc': history_resnet_50_224.history['val_accuracy']
    })
df_resnet_50_224.to_csv('./history_resnet_34_224.csv', index=False)